In [ ]:
"""
  run and restart runtime once, av library needs that for some reason, it is need 
"""
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
!pip install pytorch_lightning
import torch.utils as torchU
import pytorch_lightning as torch_light
from torchvision import datasets, models, transforms
import numpy as np
import os
from matplotlib import pyplot as plt
import math
from torch.autograd import Variable
!pip install av
import torch.nn.functional as F

In [ ]:
%cd "/content/drive/My Drive/Cuda Vision Lab Final Project"
!ls

In [ ]:
%cd "/content"
!ls

In [ ]:
"""
  copy pasted from pytorch_ssim for easier code control and ability to change
"""
def gaussian(window_size, sigma):
    gauss = torch.Tensor([math.exp(-(x - window_size/2)**2/float(2*sigma**2)) for x in range(window_size)])
    return gauss/gauss.sum()

def create_window(window_size, channel):
    _1D_window = gaussian(window_size, 1.5).unsqueeze(1)
    _2D_window = _1D_window.mm(_1D_window.t()).float().unsqueeze(0).unsqueeze(0)
    window = Variable(_2D_window.expand(channel, 1, window_size, window_size))
    return window

def _ssim(img1, img2, window, window_size, channel, size_average = True):
    int_pad = int(window_size/2)
    mu1 = F.conv2d(img1, window, padding = int_pad, groups = channel)
    mu2 = F.conv2d(img2, window, padding = int_pad, groups = channel)

    mu1_sq = mu1.pow(2)
    mu2_sq = mu2.pow(2)
    mu1_mu2 = mu1*mu2

    int_pad = int(window_size/2)
    sigma1_sq = F.conv2d(img1*img1, window, padding = int_pad, groups = channel) - mu1_sq
    sigma2_sq = F.conv2d(img2*img2, window, padding = int_pad, groups = channel) - mu2_sq
    sigma12 = F.conv2d(img1*img2, window, padding = int_pad, groups = channel) - mu1_mu2

    C1 = 0.01**2
    C2 = 0.03**2

    ssim_map = ((2*mu1_mu2 + C1)*(2*sigma12 + C2))/((mu1_sq + mu2_sq + C1)*(sigma1_sq + sigma2_sq + C2))

    if size_average:
        return ssim_map.mean()
    else:
        return ssim_map.mean(1).mean(1).mean(1)

class SSIM(torch.nn.Module):
    def __init__(self, window_size = 11, size_average = True):
        super(SSIM, self).__init__()
        self.window_size = window_size
        self.size_average = size_average
        self.channel = 1
        self.window = create_window(window_size, self.channel)

    def forward(self, img1, img2):
        (_, channel, _, _) = img1.size()

        if channel == self.channel:
            window = self.window
        else:
            window = create_window(self.window_size, channel)
            self.window = window
            self.channel = channel

        return _ssim(img1, img2, window, self.window_size, channel, self.size_average)

def ssim(img1, img2, window_size = 11, size_average = True):
    (_, channel, _, _) = img1.size()
    window = create_window(window_size, channel)
    return _ssim(img1, img2, window, window_size, channel, size_average)

In [ ]:
"""
  code is taken from following Git-repository:
  https://github.com/happyjin/ConvGRU-pytorch.git
  ConvLSTM_pytorch is released under the MIT License (refer to the LICENSE file for details).
"""
class ConvGRU(nn.Module):
  def __init__(self, input_size, input_dim, hidden_dim, kernel_size, bias, dtype, batch_size):
    """
    Initialize the ConvLSTM cell
    :param input_size: (int, int)
        Height and width of input tensor as (height, width).
    :param input_dim: int
        Number of channels of input tensor.
    :param hidden_dim: int
        Number of channels of hidden state.
    :param kernel_size: (int, int)
        Size of the convolutional kernel.
    :param bias: bool
        Whether or not to add the bias.
    :param dtype: torch.cuda.FloatTensor or torch.FloatTensor
        Whether or not to use cuda.
    """
    super(ConvGRU, self).__init__()
    self.height, self.width = input_size
    self.padding = kernel_size // 2, kernel_size // 2
    self.hidden_dim = hidden_dim
    self.bias = bias
    self.dtype = dtype
    self.init_hidden(batch_size)

    self.conv_gates = nn.Conv2d(in_channels=input_dim + hidden_dim,
                                out_channels=2*self.hidden_dim,  # for update_gate,reset_gate respectively
                                kernel_size=kernel_size,
                                padding=self.padding,
                                bias=self.bias)

    self.conv_can = nn.Conv2d(in_channels=input_dim+hidden_dim,
                          out_channels=self.hidden_dim, # for candidate neural memory
                          kernel_size=kernel_size,
                          padding=self.padding,
                          bias=self.bias)
  def init_hidden(self, batch_size):
    self.h_cur = (Variable(torch.zeros(batch_size, self.hidden_dim, self.height, self.width)).type(self.dtype))
  def forward(self, input_tensor):
    """
    :param self:
    :param input_tensor: (b, c, h, w)
        input is actually the target_model
    :param h_cur: (b, c_hidden, h, w)
        current hidden and cell states respectively
    :return: h_next,
        next hidden state
    """
    self.check_device(input_tensor)
    combined = torch.cat([input_tensor, self.h_cur], dim=1)
    combined_conv = self.conv_gates(combined)

    gamma, beta = torch.split(combined_conv, self.hidden_dim, dim=1)
    reset_gate = torch.sigmoid(gamma)
    update_gate = torch.sigmoid(beta)

    combined = torch.cat([input_tensor, reset_gate*self.h_cur], dim=1)
    cc_cnm = self.conv_can(combined)
    cnm = torch.tanh(cc_cnm)

    h_next = (1 - update_gate) * self.h_cur + update_gate * cnm
    return h_next
  def check_device(self,input_tensors):
    if self.h_cur.device != input_tensors.device:
      self.h_cur.data = self.h_cur.to(input_tensors.get_device())

In [ ]:
"""
  This module is provided from our supervisor Hafez Farazi
  related paper: http://ais.uni-bonn.de/~hfarazi/papers/LocDep.pdf
  code also provided from: https://github.com/AIS-Bonn/LocDepVideoPrediction.git
"""
class LocationAwareConv2d(torch.nn.Conv2d):
    def __init__(self,w,h,in_channels, out_channels, kernel_size, stride=1, padding=0, dilation=1, groups=1, bias=True):
        super().__init__(in_channels, out_channels, kernel_size, stride=stride, padding=padding, dilation=dilation, groups=groups, bias=bias)
        self.locationBias=torch.nn.Parameter(torch.zeros(w,h,3))
        self.locationEncode=torch.autograd.Variable(torch.ones(w,h,3))
    def forward(self,inputs):
        if self.locationBias.device != inputs.device:
            self.locationBias.data=self.locationBias.to(inputs.get_device())
        if self.locationEncode.device != inputs.device:
            self.locationEncode.data=self.locationEncode.to(inputs.get_device())
        b=self.locationBias*self.locationEncode
        return super().forward(inputs)+b[:,:,0]+b[:,:,1]+b[:,:,2]

In [ ]:
"""
  module to deal with midblocks of model 1
"""
class Paraleller(nn.Module):
  def __init__(self,list_of_modules):
    super(Paraleller, self).__init__()
    self.module_list = nn.ModuleList(list_of_modules)
  def forward(self, x):
    ans = []
    for module in self.module_list:
      ans.append(module(x))
    return ans

In [ ]:
"""
  download the dataset and annonations
"""
!wget https://www.crcv.ucf.edu/data/UCF101/UCF101.rar --no-check-certificate
!wget https://www.crcv.ucf.edu/data/UCF101/UCF101TrainTestSplits-RecognitionTask.zip --no-check-certificate

In [ ]:
"""
  unpack downloaded dataset and annnonations
"""
get_ipython().system_raw("unrar x UCF101.rar")
!unzip UCF101TrainTestSplits-RecognitionTask.zip

In [ ]:
# important constant: (frames_per_clip-3)*batchsize = true batchsize
frames_per_clip = 50
frames_between_clip = 200
inp_dim=64

In [ ]:
tfs = transforms.Compose([
             # scale in [0, 1]
             transforms.Lambda(lambda x: x / 255.),
             # reshape into (T, C, H, W)
             transforms.Lambda(lambda x: x.permute(0, 3, 1, 2) ) ,
             # resize frames
             transforms.Lambda(lambda x: F.interpolate(x,(inp_dim,inp_dim)))
    ])

In [ ]:
def loader_avi_to_tensor(path):
  frames,_,_= torchvision.io.read_video(path)
  return frames

In [ ]:
"""
  using generic DatasetLoader
"""
ufc101_dset_train = datasets.folder.DatasetFolder(  root="UCF-101",
                                                    loader=loader_avi_to_tensor,
                                                    extensions=("avi"),
                                                    transform=tfs                                             
                                                  )

In [ ]:
print(len(ufc101_dset_train))

13320


In [ ]:
loader = torchU.data.DataLoader(ufc101_dset_train,batch_size=1,collate_fn=custom_collate)
rnd_number = int(torch.randint(10,size=(1,)))
print(rnd_number)
counter=1
chosen_batch = None
chosen_labels = None
for batches in loader:
  if counter==rnd_number:
    chosen_batch = batches
    break
  counter+=1

9


/usr/local/lib/python3.6/dist-packages/torchvision/io/video.py:106: UserWarning: The pts_unit 'pts' gives wrong results and will be removed in a follow-up version. Please use pts_unit 'sec'.
  + "follow-up version. Please use pts_unit 'sec'."


In [ ]:
for video,label in chosen_batch:
  print(video.shape,label)

torch.Size([216, 3, 64, 64]) 0


In [ ]:
def custom_collate(batch):
    filtered_batch = []
    for video in batch:
        filtered_batch.append(video)
    return filtered_batch

In [ ]:
def calculate_split(dataset_len,batch_size):
  len1 = int(dataset_len*0.6)
  while len1%batch_size!=0:
    len1-=len1%batch_size
  if dataset_len-len1%2==0:
    len2 = (dataset_len-len1)//2
    len3 = len2
  else:
    len2 = (dataset_len-len1-1)//2
    len3 = dataset_len-len1-len2
  return len1,len2,len3

In [ ]:
class pred_Project(torch_light.LightningModule):
  def __init__(self, hparams):
    super(pred_Project, self).__init__()
    self.hparams = hparams
    
    """
      load pretrained resnet18
    """
    self.resnet = models.resnet18(pretrained=True)
    
    """
      some constants of the network
    """
    inp_size = self.hparams["input_dim"] #assumption number is even and power of 2
    i2 = int(inp_size/2)
    i4 = int(inp_size/4)
    i8 = int(inp_size/8)
    i16 = int(inp_size/16)
    true_batchsize = self.hparams["batch_size"]
    """
      create a partial layer of the proposed network and structure them in packs
    """
    #BN+Relu+Maxpool
    self.pre_block = nn.Sequential(
        self.resnet.bn1,
        self.resnet.relu,
        self.resnet.maxpool
    )
    #middle row block 1
    self.midblock1 = Paraleller(
        [nn.Conv2d(64,64,1,1),
          ConvGRU((i2,i2),64,64,kernel_size=3,bias=False,dtype=torch.FloatTensor,batch_size=true_batchsize),
          ConvGRU((i2,i2),64,64,kernel_size=5,bias=False,dtype=torch.FloatTensor,batch_size=true_batchsize),
          ConvGRU((i2,i2),64,64,kernel_size=7,bias=False,dtype=torch.FloatTensor,batch_size=true_batchsize)  
        ]
    )
    #middle row block 2
    self.midblock2 = Paraleller(
        [nn.Conv2d(64,64,1,1),
          ConvGRU((i4,i4),64,64,kernel_size=3,bias=False,dtype=torch.FloatTensor,batch_size=true_batchsize),
          ConvGRU((i4,i4),64,64,kernel_size=5,bias=False,dtype=torch.FloatTensor,batch_size=true_batchsize),
          ConvGRU((i4,i4),64,64,kernel_size=7,bias=False,dtype=torch.FloatTensor,batch_size=true_batchsize)
        ]
    )
    #middle row block 3
    self.midblock3 = Paraleller(
        [LocationAwareConv2d(w=i8,h=i8,in_channels=128,out_channels=64,kernel_size=1),
          ConvGRU((i8,i8),128,64,kernel_size=3,bias=False,dtype=torch.FloatTensor,batch_size=true_batchsize),
          ConvGRU((i8,i8),128,64,kernel_size=5,bias=False,dtype=torch.FloatTensor,batch_size=true_batchsize),
          ConvGRU((i8,i8),128,64,kernel_size=7,bias=False,dtype=torch.FloatTensor,batch_size=true_batchsize)
        ]
    )
    #middle row block 4
    self.midblock4 = Paraleller(
        [LocationAwareConv2d(w=i16,h=i16,in_channels=256,out_channels=64,kernel_size=1),
          ConvGRU((i16,i16),256,64,kernel_size=3,bias=False,dtype=torch.FloatTensor,batch_size=true_batchsize),
          ConvGRU((i16,i16),256,64,kernel_size=5,bias=False,dtype=torch.FloatTensor,batch_size=true_batchsize),
          ConvGRU((i16,i16),256,64,kernel_size=7,bias=False,dtype=torch.FloatTensor,batch_size=true_batchsize)
        ]
    )
    #left block block 1
    self.leftblock1 = nn.Sequential(
        nn.ReLU(),
        nn.Conv2d(512,1024,3,1,1),
        nn.PixelShuffle(2),
        nn.Conv2d(256,64,3,1,1)
    )
    #left block block 2
    self.leftblock2 = nn.Sequential(
        nn.ReLU(),
        nn.Conv2d(320,1024,3,1,1),
        nn.PixelShuffle(2),
        nn.Conv2d(256,64,3,1,1)
    )
    #left block block 3
    self.leftblock3 = nn.Sequential(
        nn.ReLU(),
        nn.Conv2d(320,1024,3,1,1),
        nn.PixelShuffle(2),
        nn.Conv2d(256,64,3,1,1)
    )
    #left block block 4
    self.leftblock4 = nn.Sequential(
        nn.ReLU(),
        nn.Conv2d(320,1024,3,1,1),
        nn.PixelShuffle(2),
        nn.Conv2d(256,64,3,1,1)
    )
    # last layer
    self.lucky_layer = nn.Conv2d(320,3,1,1,0)
  def forward(self, x):
      x = self.resnet.conv1(x)
      self.feed_to_midblock1 = x.clone()
      x = self.pre_block(x)
      x = self.resnet.layer1(x)
      self.feed_to_midblock2 = x.clone()
      x = self.resnet.layer2(x)
      self.feed_to_midblock3 = x.clone()
      x = self.resnet.layer3(x)
      self.feed_to_midblock4 = x.clone()
      x = self.resnet.layer4(x)
      
      x = self.leftblock1(x)
      y4 = self.midblock4(self.feed_to_midblock4)
      y4.append(x)
      x = torch.cat(y4,dim=1)

      x = self.leftblock2(x)
      y3 = self.midblock3(self.feed_to_midblock3)
      y3.append(x)
      x = torch.cat(y3,dim=1)

      x=self.leftblock3(x)
      y2 = self.midblock2(self.feed_to_midblock2)
      y2.append(x)
      x = torch.cat(y2,dim=1)

      x=self.leftblock4(x)
      y1 = self.midblock1(self.feed_to_midblock1)
      y1.append(x)
      x =torch.cat(y1,dim=1)

      return self.lucky_layer(x)      
  def training_step(self, batch, batch_nb):
      inp_dim = self.hparams["input_dim"]
      data,_ = batch
      if data.shape[0] != self.hparams["batch_size"]:
        print("(train)Not full batchsize:",data.shape[0],"!=",self.hparams["batch_size"])
        A = torch.rand((1,3,32,32)).cuda()
        B = calculate_full_loss(A,A,hparams)-1
        return {"loss": B}
      batch,labels = build_batch_and_labels(data,inp_dim)
      temp_tensor = torch.rand((1,3,32,32)).cuda()
      # cumulated loss
      cum_loss = calculate_full_loss(temp_tensor,temp_tensor,hparams)-1
      for b,l in zip(batch,labels):
        pred = self.forward(b)
        loss,loss_dic = calculate_full_loss(pred,l,self.hparams)
        cum_loss += loss
      return {"loss": cum_loss}
  def training_epoch_end(self, outputs):
      avg_loss = torch.stack([x['loss'] for x in outputs]).mean()
      logs = {'train/loss': avg_loss, 'step': self.current_epoch}
      return {'train_avg_loss': avg_loss, 'log': logs}
  def validation_step(self, batch, batch_idx):
      inp_dim = self.hparams["input_dim"]
      data,_ = batch
      if data.shape[0] != self.hparams["batch_size"]:
        print("(val)Not full batchsize:",data.shape[0],"!=",self.hparams["batch_size"])
        A = torch.rand((1,3,32,32)).cuda()
        B = calculate_full_loss(A,A,hparams)-1
        return {"val_loss": B}
      batch,labels = build_batch_and_labels(data,inp_dim)
      temp_tensor = torch.rand((1,3,32,32)).cuda()
      cum_loss = calculate_full_loss(temp_tensor,temp_tensor,hparams)-1
      for b,l in zip(batch,labels):
        pred = self.forward(b)
        loss,loss_dic = calculate_full_loss(pred,l,self.hparams)
        cum_loss += loss
      return {"val_loss": cum_loss}
  def validation_epoch_end(self, outputs):
      avg_loss = torch.stack([x['val_loss'] for x in outputs]).mean()
      logs = {'val/loss': avg_loss, 'step': self.current_epoch}
      return {'val_avg_loss': avg_loss, 'log': logs}
  def test_step(self, batch, batch_idx):
      inp_dim = self.hparams["input_dim"]
      data,_ = batch

      batch,labels = build_batch_and_labels(data,inp_dim)
      pred = self.forward(batch)
      loss = calculate_full_loss(pred,labels,self.hparams)
      loss =  l1+ l2 + l3
      return{"test_loss":loss}
  def test_epoch_end(self, outputs):
      avg_loss = torch.stack([x['test_loss'] for x in outputs]).mean()
      logs = {'test/loss': avg_loss, 'step': self.current_epoch}
      return {'test_avg_loss': avg_loss, 'log': logs}
  def prepare_data(self):
      len1, len2 = calculate_split(len(ufc101_dset_train),hparams["batch_size"])
      print(len1,len2)
      self.train_dset, self.val_dset = torchU.data.random_split(ufc101_dset_train,[len1,len2])
  def train_dataloader(self):
      return torchU.data.DataLoader(self.train_dset,hparams["batch_size"],shuffle=True,collate_fn=custom_collate,num_workers=4)
  def val_dataloader(self):
      return torchU.data.DataLoader(self.val_dset,hparams["batch_size"],shuffle=False,collate_fn=custom_collate,num_workers=4)
  def test_dataloader(self):
      return torchU.data.DataLoader(self.val_dset,hparams["batch_size"],shuffle=False,collate_fn=custom_collate,num_workers=4)
  def configure_optimizers(self):
      optimizer = None
      if self.hparams["optimizer"] == 'sgd':
          optimizer = torch.optim.SGD(self.parameters(), lr=self.hparams["learning_rate"], momentum=0.8,
                                      nesterov=True)
      elif self.hparams["optimizer"] == 'adam':
          optimizer = torch.optim.Adam(self.parameters(), lr=self.hparams["learning_rate"])
      elif self.hparams["optimizer"] == 'adag':
          optimizer = torch.optim.Adagrad(self.parameters(), lr=self.hparams["learning_rate"])
      elif self.hparams["optimizer"] == 'adad':
          optimizer = torch.optim.Adadelta(self.parameters(), lr=self.hparams["learning_rate"])
      elif self.hparams["optimizer"] == 'rmsp':
          optimizer = torch.optim.RMSprop(self.parameters(), lr=self.hparams["learning_rate"], momentum=0.8)
      lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1, 0.98)
      return [optimizer], [lr_scheduler]
  def reset_convGRU(self,batch_size):
    for i in range(len(self.midblock1.module_list)):
      if i==0:
        continue
      self.midblock1.module_list[i].init_hidden(batch_size)
      self.midblock2.module_list[i].init_hidden(batch_size)
      self.midblock3.module_list[i].init_hidden(batch_size)
      self.midblock4.module_list[i].init_hidden(batch_size)

In [ ]:
"""
  prepare model specification for training
"""
epoches=1
torch_light.seed_everything()

"""
  hparams
"""
hparams={}
hparams["optimizer"]="adam"
hparams["learning_rate"]=0.001
hparams["batch_size"]=16
batch_size=hparams["batch_size"]
hparams["input_dim"]=inp_dim
hparams["model"]= pred_Project.load_from_checkpoint("model_01.ptl")
hparams["model"] = hparams["model"].cuda()
hparams["loss_func"]="crossent"
hparams["lin_dim"] = [pow(2,9),pow(2,8)]

No correct seed found, seed set to 595414502


bceloss oder binarycrossentropy

In [ ]:
class class_Project(torch_light.LightningModule):
  def __init__(self, hparams):
    super(class_Project, self).__init__()
    self.hparams = hparams
    self.trained_net = hparams["model"]
    self.trained_net.reset_convGRU(1)
    self.lin_dim = hparams["lin_dim"]

    _,shapes = self.get_hidden_states()
    self.linear_layers = self.create_linear_layers(shapes)
    self.final_linear_layer = nn.Linear(self.lin_dim[1]*4*3,101)
  def forward(self, x):
    collector = []
    hidden_states,_ = self.get_hidden_states()
    for i,block in enumerate(hidden_states):
      for j,state in enumerate(block):
        collector.append((self.linear_layers[i][j])(state))
    collector = torch.cat(collector,dim=0)
    collector = collector.view(1,-1)
    return self.final_linear_layer(collector)
  def training_step(self, batch, batch_nb):
    pred = []
    labels = []
    for video,label in batch:
      self.trained_net.reset_convGRU(1)
      for frame in video:
        _ = self.trained_net(frame.unsqueeze(0))
      pred.append(self.forward(None))
      labels.append(torch.Tensor([label]))
    pred = torch.stack(pred)
    pred = pred.view(pred.shape[0],pred.shape[2])
    labels = torch.stack(labels)
    labels = labels.type(torch.LongTensor)
    labels = (labels.view(labels.shape[0])).cuda()
    loss_func = self.get_loss_func()
    loss = loss_func(pred,labels)
    acc = torch.sum(torch.max(pred, 1)[1] == labels).float() / len(labels)
    return {"loss": loss,"train/acc":acc}
  def training_epoch_end(self, outputs):
    avg_loss = torch.stack([x['loss'] for x in outputs]).mean()
    avg_acc = torch.stack([x['train/acc'] for x in outputs]).mean()
    logs = {'train/loss': avg_loss, 'step': self.current_epoch,'train/acc':avg_acc}
    return {'train_avg_loss': avg_loss, 'log': logs}
  def validation_step(self, batch, batch_idx):
    pred = []
    labels = []
    for video,label in batch:
      self.trained_net.reset_convGRU(1)
      for frame in video:
        _ = self.trained_net(frame.unsqueeze(0))
      pred.append(self.forward(None))
      labels.append(torch.Tensor([label]))
    pred = torch.stack(pred)
    pred = pred.view(pred.shape[0],pred.shape[2])
    labels = torch.stack(labels)
    labels = labels.type(torch.LongTensor)
    labels = (labels.view(labels.shape[0])).cuda()
    loss_func = self.get_loss_func()
    loss = loss_func(pred,labels)
    acc = torch.sum(torch.max(pred, 1)[1] == labels).float() / len(labels)
    return {"val_loss": loss,"val/acc":acc}
  def validation_epoch_end(self, outputs):
    avg_loss = torch.stack([x['val_loss'] for x in outputs]).mean()
    avg_acc = torch.stack([x['val/acc'] for x in outputs]).mean()
    logs = {'val/loss': avg_loss, 'step': self.current_epoch,'val/acc':avg_acc}
    return {'val_avg_loss': avg_loss, 'log': logs}
  def test_step(self, batch, batch_idx):
    pred = []
    labels = []
    for video,label in batch:
      self.trained_net.reset_convGRU(1)
      for frame in video:
        _ = self.trained_net(frame.unsqueeze(0))
      pred.append(self.forward(None))
      labels.append(torch.Tensor([label]))
    pred = torch.stack(pred)
    pred = pred.view(pred.shape[0],pred.shape[2])
    labels = torch.stack(labels)
    labels = labels.type(torch.LongTensor)
    labels = (labels.view(labels.shape[0])).cuda()
    loss_func = self.get_loss_func()
    loss = loss_func(pred,labels)
    acc = torch.sum(torch.max(pred, 1)[1] == labels).float() / len(labels)
    return {"test_loss": loss,"test/acc":acc}
  def test_epoch_end(self, outputs):
    avg_loss = torch.stack([x['test_loss'] for x in outputs]).mean()
    avg_acc = torch.stack([x['test/acc'] for x in outputs]).mean()
    logs = {'test/loss': avg_loss, 'step': self.current_epoch,'test/acc':avg_acc}
    return {'test_avg_loss': avg_loss, 'log': logs}
  def prepare_data(self):
    len1, len2,len3 = calculate_split(len(ufc101_dset_train),hparams["batch_size"])
    print(len1,len2,len3)
    self.train_dset, self.val_dset,self.test_dset = torchU.data.random_split(ufc101_dset_train,[len1,len2,len3])
  def train_dataloader(self):
    return torchU.data.DataLoader(self.train_dset,hparams["batch_size"],shuffle=True,collate_fn=custom_collate,num_workers=4)
  def val_dataloader(self):
    return torchU.data.DataLoader(self.val_dset,hparams["batch_size"],shuffle=False,collate_fn=custom_collate,num_workers=4)
  def test_dataloader(self):
    return torchU.data.DataLoader(self.test_dset,hparams["batch_size"],shuffle=False,collate_fn=custom_collate,num_workers=4)
  def configure_optimizers(self):
    optimizer = None
    if self.hparams["optimizer"] == 'sgd':
        optimizer = torch.optim.SGD(self.parameters(), lr=self.hparams["learning_rate"], momentum=0.8,
                                    nesterov=True)
    elif self.hparams["optimizer"] == 'adam':
        optimizer = torch.optim.Adam(self.parameters(), lr=self.hparams["learning_rate"])
    elif self.hparams["optimizer"] == 'adag':
        optimizer = torch.optim.Adagrad(self.parameters(), lr=self.hparams["learning_rate"])
    elif self.hparams["optimizer"] == 'adad':
        optimizer = torch.optim.Adadelta(self.parameters(), lr=self.hparams["learning_rate"])
    elif self.hparams["optimizer"] == 'rmsp':
        optimizer = torch.optim.RMSprop(self.parameters(), lr=self.hparams["learning_rate"], momentum=0.8)
    lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1, 0.98)
    return [optimizer], [lr_scheduler]
  def get_loss_func(self):
    if hparams["loss_func"]=="bce":
      return nn.BCELoss()
    if hparams["loss_func"]=="crossent":
      return nn.CrossEntropyLoss()
  def get_hidden_states(self):
    block1 = self.trained_net.midblock1.module_list[1:]
    block2 = self.trained_net.midblock2.module_list[1:]
    block3 = self.trained_net.midblock3.module_list[1:]
    block4 = self.trained_net.midblock4.module_list[1:]

    hidden_states = []
    hidden_states1 = [x.h_cur.view(x.h_cur.shape[0],-1) for x in block1]
    hidden_states2 = [x.h_cur.view(x.h_cur.shape[0],-1) for x in block2]
    hidden_states3 = [x.h_cur.view(x.h_cur.shape[0],-1) for x in block3]
    hidden_states4 = [x.h_cur.view(x.h_cur.shape[0],-1) for x in block4]

    hidden_states.append(hidden_states1)
    hidden_states.append(hidden_states2)
    hidden_states.append(hidden_states3)
    hidden_states.append(hidden_states4)

    shapes = []
    shapes1 = [x.shape[1] for x in hidden_states1]
    shapes2 = [x.shape[1] for x in hidden_states2]
    shapes3 = [x.shape[1] for x in hidden_states3]
    shapes4 = [x.shape[1] for x in hidden_states4]

    shapes.append(shapes1)
    shapes.append(shapes2)
    shapes.append(shapes3)
    shapes.append(shapes4)
    return hidden_states,shapes
  def create_linear_layers(self,shapes):
    layer_list = [[],[],[],[]]
    for i,block in enumerate(shapes):
      for j,GRU in enumerate(block):
        tmp = nn.Sequential(
            nn.Linear(GRU,self.lin_dim[0]),
            nn.Sigmoid(),
            nn.Linear(self.lin_dim[0],self.lin_dim[1])
        )
        layer_list[i].append(tmp)
      layer_list[i] = nn.ModuleList(layer_list[i])
    return nn.ModuleList(layer_list)

In [ ]:
"""
  initialize net and trainer
"""
net_p2 = class_Project(hparams)
trainer= torch_light.Trainer(max_epochs=epoches,gpus=-1,fast_dev_run=False,checkpoint_callback=False)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
CUDA_VISIBLE_DEVICES: [0]


In [ ]:
"""
  Run training on model
"""
trainer.fit(net_p2)

/usr/local/lib/python3.6/dist-packages/pytorch_lightning/utilities/distributed.py:37: UserWarning: Could not log computational graph since the `model.example_input_array` attribute is not set or `input_array` was not given
  warnings.warn(*args, **kwargs)

  | Name               | Type         | Params
----------------------------------------------------
0 | trained_net        | pred_Project | 38 M  
1 | linear_layers      | ModuleList   | 135 M 
2 | final_linear_layer | Linear       | 310 K 


7984 2667 2669


/usr/local/lib/python3.6/dist-packages/torchvision/io/video.py:106: UserWarning: The pts_unit 'pts' gives wrong results and will be removed in a follow-up version. Please use pts_unit 'sec'.
  + "follow-up version. Please use pts_unit 'sec'."
/usr/local/lib/python3.6/dist-packages/torchvision/io/video.py:106: UserWarning: The pts_unit 'pts' gives wrong results and will be removed in a follow-up version. Please use pts_unit 'sec'.
  + "follow-up version. Please use pts_unit 'sec'."
/usr/local/lib/python3.6/dist-packages/torchvision/io/video.py:106: UserWarning: The pts_unit 'pts' gives wrong results and will be removed in a follow-up version. Please use pts_unit 'sec'.
  + "follow-up version. Please use pts_unit 'sec'."
/usr/local/lib/python3.6/dist-packages/torchvision/io/video.py:106: UserWarning: The pts_unit 'pts' gives wrong results and will be removed in a follow-up version. Please use pts_unit 'sec'.
  + "follow-up version. Please use pts_unit 'sec'."


/usr/local/lib/python3.6/dist-packages/torchvision/io/video.py:106: UserWarning: The pts_unit 'pts' gives wrong results and will be removed in a follow-up version. Please use pts_unit 'sec'.
  + "follow-up version. Please use pts_unit 'sec'."
/usr/local/lib/python3.6/dist-packages/torchvision/io/video.py:106: UserWarning: The pts_unit 'pts' gives wrong results and will be removed in a follow-up version. Please use pts_unit 'sec'.
  + "follow-up version. Please use pts_unit 'sec'."
/usr/local/lib/python3.6/dist-packages/torchvision/io/video.py:106: UserWarning: The pts_unit 'pts' gives wrong results and will be removed in a follow-up version. Please use pts_unit 'sec'.
  + "follow-up version. Please use pts_unit 'sec'."
/usr/local/lib/python3.6/dist-packages/torchvision/io/video.py:106: UserWarning: The pts_unit 'pts' gives wrong results and will be removed in a follow-up version. Please use pts_unit 'sec'.
  + "follow-up version. Please use pts_unit 'sec'."


In [ ]:
"""
  Save model as checkpoint
"""

trainer.save_checkpoint("model_c_01.ptl")

In [ ]:
"""
  Run test on model
"""
trainer.test(net_p2)

In [ ]:
%reload_ext tensorboard
#%cd "/content/drive/My Drive/Cuda Vision Lab Final Project"
%cd "/content"
!ls

In [ ]:
%tensorboard --logdir lightning_logs